In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import random
import csv
from varname import nameof

In [4]:
#By Eshan - make sure input rates are floats
def gen_fitness(self,allele,conc,drugless_rate,ic50):        
    c = -.6824968 # empirical curve fit
    # logistic equation from Ogbunugafor 2016
    conc = conc/10**6 # concentration in uM, convert to M
    # ic50 is already log-ed in the dataset
    log_eqn = lambda d,i: d/(1+np.exp((i-np.log10(conc))/c))
    if conc <= 0:
        fitness = drugless_rate[allele]
    else:
        print(drugless_rate[allele], ic50[allele])
        fitness = log_eqn(drugless_rate[allele],ic50[allele])
    return(fitness)


In [5]:
def glcm_landscape_greylevelinv(lscp, method=1, levels=8, normalise="TRUE"):
    
    N = len(lscp)
    lscp_scale = np.interp(lscp, (min(lscp), max(lscp)), (0, +1))
    
    #Generate quantized values that landscape will be mapped to
    scale = []
    for i in range(levels):
        scale.append(i/(levels-1))
    
    if method==1:
    #Map each value to scaled 
        inds = np.digitize(lscp_scale,scale)
        inds = inds-1
            
    if method==2:
        inds = []
        for x in lscp_scale:    
            y=quantize(x, scale)
            y=int(y*(levels-1))
            inds.append(y)
        
    #print(lscp_scale)
    #print(inds)
    
    #Generate list of neighbours
    hammy = np.array([0,0,0]) 
    for i in range(0,len(lscp)):
        for j in range(i,len(lscp)):
            ob1 = Solution()
            ham = ob1.hammingDistance(i,j)
            if ob1.hammingDistance(i,j)==1:
                hammy=np.vstack([hammy, [i, j, ham]])
    
    #Initialize GLCM        
    hist = np.zeros((len(scale),len(scale)))
    hammy = np.delete(hammy, (0), axis=0)
    #Populate
    for row in hammy:
        x,y,z=row
        hist[inds[x], inds[y]]=hist[inds[x], inds[y]]+1
    
    N=len(scale)
    delij = N**2
    
    #Normalized GLCM
    if (normalise=="TRUE"):
        hist = hist/(np.sum(hist)*delij)
    
    
    energy = np.sum(hist*hist)*delij
    
    entropy=0
    contrast=0
    homogeneity=0
    correlation = 0
    autocorrelation = 0
    mu = 0
    var = 0
    
    for i in range(0, len(scale)):
        for j in range(0, len(scale)):
            mu = mu + (i/N)*hist[i,j]*(1/N)

    
    for i in range(0, len(scale)):
        for j in range(0, len(scale)):
            var = var + (1/N)*hist[i,j]*(i/N-mu)**2
    
    myDict = {}

    for i in range(0, len(scale)):
        for j in range(0, len(scale)):
            autocorrelation = autocorrelation+ i*j*hist[i,j]
            entropy = entropy - hist[i,j]*np.log(hist[i,j]+0.00000001)*delij
            contrast = contrast + (i/N-j/N)*(i/N-j/N)*hist[i,j]*delij
            homogeneity = homogeneity + hist[i,j]*delij/(1+(i/N-j/N)**2)
            correlation = correlation + hist[i,j]*(i/N-mu)*(j/N-mu)/var
            
    N_loci = int(math.log(N,2))
    TM = get_TM(inds,N=N_loci)
    TM_diff = np.sum(abs(get_TM(inds, N=N_loci) - get_TM(lscp, N=N_loci)))
    
    myDict = {"TM":TM, "TM_diff":TM_diff, "autocorrelation": autocorrelation, "levels":levels, "method":method,"energy":energy, "correlation":correlation, "entropy":entropy,"contrast":contrast, "homogeneity":homogeneity}
    #return(energy, entropy, contrast, homogeneity)
    return(myDict)

In [6]:


with open('landscapes/pyrimethamine_ic50.csv', newline='') as f:
    reader = csv.reader(f)
    pyr_ic50 = list(reader)[0]
    pyr_ic50 = [float(i) for i in pyr_ic50]

    
with open('landscapes/cycloguanil_ic50.csv', newline='') as f:
    reader = csv.reader(f)
    cyc_ic50 = list(reader)[0]
    cyc_ic50 = [float(i) for i in cyc_ic50]
    
with open('landscapes/ogbunugafor_drugless.csv', newline='') as f:
    reader = csv.reader(f)
    drugless = list(reader)[0]
    drugless = [float(i) for i in drugless]
    


In [7]:
#Evaluate this concentration option   5** or other number to the power to create even intervals on log scale?
concentrations = [10**(x/2) for x in range(-6,10)]    

pyr_fitness = {}
for i in concentrations:
    list_fit = list()
    for j in range(0,16):
        list_fit.append(gen_fitness(0,j,i,drugless, pyr_ic50))
    pyr_fitness[i]=list_fit
    
cyc_fitness = {}
for i in concentrations:
    list_fit = list()
    for j in range(0,16):
        list_fit.append(gen_fitness(0,j,i,drugless, cyc_ic50))
    cyc_fitness[i]=list_fit

    
pyr_fitness2 = {}
for i in range(0,16):
    list_fit = list()
    for j in concentrations:
        list_fit.append(gen_fitness(0,i,j,drugless, pyr_ic50))
    pyr_fitness2[i]=list_fit
    
cyc_fitness2 = {}
for i in range(0,16):
    list_fit = list()
    for j in concentrations:
        list_fit.append(gen_fitness(0,i,j,drugless, cyc_ic50))
    cyc_fitness2[i]=list_fit

1.398 -6.286
1.275 -5.812
1.227 -4.239
0.0 0.0
1.37 -6.046
1.375 -5.774
1.397 -3.732
1.219 -3.55
1.119 -5.724
1.184 -5.491
1.306 -4.015
1.0 -4.6
1.273 -5.773
1.282 -5.624
1.45 -3.587
1.25 -3.3
1.398 -6.286
1.275 -5.812
1.227 -4.239
0.0 0.0
1.37 -6.046
1.375 -5.774
1.397 -3.732
1.219 -3.55
1.119 -5.724
1.184 -5.491
1.306 -4.015
1.0 -4.6
1.273 -5.773
1.282 -5.624
1.45 -3.587
1.25 -3.3
1.398 -6.286
1.275 -5.812
1.227 -4.239
0.0 0.0
1.37 -6.046
1.375 -5.774
1.397 -3.732
1.219 -3.55
1.119 -5.724
1.184 -5.491
1.306 -4.015
1.0 -4.6
1.273 -5.773
1.282 -5.624
1.45 -3.587
1.25 -3.3
1.398 -6.286
1.275 -5.812
1.227 -4.239
0.0 0.0
1.37 -6.046
1.375 -5.774
1.397 -3.732
1.219 -3.55
1.119 -5.724
1.184 -5.491
1.306 -4.015
1.0 -4.6
1.273 -5.773
1.282 -5.624
1.45 -3.587
1.25 -3.3
1.398 -6.286
1.275 -5.812
1.227 -4.239
0.0 0.0
1.37 -6.046
1.375 -5.774
1.397 -3.732
1.219 -3.55
1.119 -5.724
1.184 -5.491
1.306 -4.015
1.0 -4.6
1.273 -5.773
1.282 -5.624
1.45 -3.587
1.25 -3.3
1.398 -6.286
1.275 -5.812
1.227 -4.

1.273 -6.663
1.273 -6.663
1.273 -6.663
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.282 -5.925
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.45 -4.395
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85
1.25 -2.85


In [8]:
from GLCM_functions import *

In [9]:
print(glcm_landscape_greylevelinv(pyr_fitness2[1], levels=8)["autocorrelation"])
print(glcm_landscape_greylevelinv(pyr_fitness2[1], levels=7)["autocorrelation"])
print(glcm_landscape_greylevelinv(pyr_fitness2[1], levels=6)["autocorrelation"])

#print(glcm_landscape_general(pyr_fitness2[1], levels=8)["autocorrelation"])
#print(glcm_landscape_general(pyr_fitness2[1], levels=7)["autocorrelation"])
#print(glcm_landscape_general(pyr_fitness2[1], levels=6)["autocorrelation"])

0.16748046875
0.15306122448979592
0.1267361111111111
